# Greenhouse Model

## Import dependencies and data

In [1]:
# import dependencies
import os
import pickle
import json
from collections import deque
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import clear_output

In [2]:
from data_funcs import k_fold_data_validation
from train_funcs import train_network
from networks import feedForwardNeuralNetwork, recurrentNeuralNetwork
from opts import AdaGrad, RMSProp, Adam

## Create data for specific fold

In [3]:
# define relevant paths
home_path = os.path.dirname(os.getcwd())
data_path = home_path + '\\data\\'
results_path = home_path + '\\nn_models\\results\\'

# get merged data
data = pd.read_csv(
    data_path + 'data_processed.csv',
    header=[0, 1],
    index_col=[0, 1, 2, 3, 4]
)

In [4]:
# define data specs
k_frac = 0.05
m = 5
t_steps = 1
n_steps = 6

# Define cols to use
cols = [
    ('temperatures', 'TA01_GT10X_GM10X'),
    #('temperatures', 'TA01_GT401_GM401_scaled'),
    #('temperatures', 'DC_GT301_damped_scaled'),
    ('temperatures', 'TA01_GT401_GM401'),
    ('temperatures', 'DC_GT301_damped'),
    ('sun', 'gsi'),
    ('power', 'phase'),
    ('time', 'minofday'),
    ('state', 'TA01_output'),
    ('state', 'TA02_output')
]

## K-fold CV

In [5]:
version = 0

In [6]:
# training hyperparams
k1 = 7
k2 = 1
lambd = 0.01
sigma = 1.0
seed = 1

# more params
n_epochs = 50
n_batch = 128
lr = 0.001

In [7]:
for k_idx in range(20):
    train_tup, test_tup, val_tup, col_params = k_fold_data_validation(
        data=data.copy(), 
        k_idx=k_idx, 
        k_frac=k_frac, 
        m=m, 
        cols=cols, 
        t_steps=t_steps, 
        n_steps=n_steps,
        setpoint=True,
        shuffle=False
    )
    
    # extract tuples
    (sequences_train, targets_train, temps_train, temps_t_train, _) = train_tup
    (sequences_test, targets_test, temps_test, temps_t_test, sequences_masked, _) = test_tup
    (sequences_val, targets_val, temps_val, temps_t_val, _) = val_tup
    
    ### ----- FEED-FORWARD NN -----
    # define network
    model_name = 'FFNN_v{}_k{}'.format(version, k_idx)
    units=[32, 16, 8]
    model = feedForwardNeuralNetwork(
        k1=k1,
        k2=k2,
        m=units,
        seed=seed
    )
    
    # define optimizer
    adam = Adam(
        beta1=0.9,
        beta2=0.999,
        eps=1e-8,
        weights=model.weights,
    )
    
    # set optimizer
    model.optimizer = adam
    
    # train model
    results = train_network(
        model=model,
        train_data=(sequences_train[:, -2, :], temps_t_train, targets_train),
        val_data=(sequences_val[:, -2, :], temps_t_val, targets_val),
        seed=seed,
        n_epochs=n_epochs,
        n_batch=n_batch,
        lambd=lambd,
        sigma=sigma,
        lr=lr,
        optimizer='adam'
    )
    
    # save results
    save_path = results_path + 'ffnn\\' + model_name + '.pickle'
    with open(save_path, 'wb') as fo:
        pickle.dump(results, fo)
    
    # clear output
    clear_output()
    
    
    ### ----- RECURRENT NN -----
    # define network
    model_name = 'RNN_v{}_k{}'.format(version, k_idx)
    units = 32
    model = recurrentNeuralNetwork(
        k1=k1,
        k2=k2,
        m=units,
        seed=seed
    )
    
    # define optimizer
    adam = Adam(
        beta1=0.9,
        beta2=0.999,
        eps=1e-8,
        weights=model.weights,
    )
    
    # set optimizer
    model.optimizer = adam
    
    # train model
    results = train_network(
        model=model,
        train_data=(sequences_train, temps_t_train, targets_train),
        val_data=(sequences_val, temps_t_val, targets_val),
        seed=seed,
        n_epochs=n_epochs,
        n_batch=n_batch,
        lambd=lambd,
        sigma=sigma,
        lr=lr,
        optimizer='adam'
    )
 
    # save results
    save_path = results_path + 'rnn\\' + model_name + '.pickle'
    with open(save_path, 'wb') as fo:
        pickle.dump(results, fo)

    # clear output
    clear_output()
    

---	 Initializing training 	---

EPOCH 0 - train loss: 2.93, val loss: 3.51
EPOCH 1 (0.60 s) - train loss: 0.65, val loss: 0.70
EPOCH 2 (0.67 s) - train loss: 0.54, val loss: 0.57
EPOCH 3 (0.61 s) - train loss: 0.47, val loss: 0.49
EPOCH 4 (0.67 s) - train loss: 0.42, val loss: 0.44
EPOCH 5 (0.67 s) - train loss: 0.38, val loss: 0.42


KeyboardInterrupt: 